# DOC2VEC Tryout 

### This file is a tryout of Word2vec method. Code Reference : 
    - https://towardsdatascience.com/multi-class-text-classification-with-doc2vec-logistic-regression-9da9947b43f4

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#DOC2VEC-Tryout" data-toc-modified-id="DOC2VEC-Tryout-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>DOC2VEC Tryout</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#This-file-is-a-tryout-of-Word2vec-method.-Code-Reference-:" data-toc-modified-id="This-file-is-a-tryout-of-Word2vec-method.-Code-Reference-:-1.0.1"><span class="toc-item-num">1.0.1&nbsp;&nbsp;</span>This file is a tryout of Word2vec method. Code Reference :</a></span></li></ul></li><li><span><a href="#1.-Clean-Up-Datafame" data-toc-modified-id="1.-Clean-Up-Datafame-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>1. Clean Up Datafame</a></span><ul class="toc-item"><li><span><a href="#Examing-Content" data-toc-modified-id="Examing-Content-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Examing Content</a></span></li></ul></li><li><span><a href="#Text-Processing" data-toc-modified-id="Text-Processing-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Text Processing</a></span></li><li><span><a href="#Regular-Train-Test-Split" data-toc-modified-id="Regular-Train-Test-Split-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Regular Train Test Split</a></span><ul class="toc-item"><li><span><a href="#Set-up-Doc2Vec-Training-with-Stopwords" data-toc-modified-id="Set-up-Doc2Vec-Training-with-Stopwords-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Set-up Doc2Vec Training with Stopwords</a></span></li><li><span><a href="#Evaluation-Models" data-toc-modified-id="Evaluation-Models-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Evaluation Models</a></span></li></ul></li><li><span><a href="#Set-up-Doc2Vec-Training-with-Stopwords-and-Time-series-Train-Test-Split" data-toc-modified-id="Set-up-Doc2Vec-Training-with-Stopwords-and-Time-series-Train-Test-Split-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Set-up Doc2Vec Training with Stopwords and Time-series Train Test Split</a></span><ul class="toc-item"><li><span><a href="#Evaluation-Models-with-Time-series-train-test-split" data-toc-modified-id="Evaluation-Models-with-Time-series-train-test-split-1.4.1"><span class="toc-item-num">1.4.1&nbsp;&nbsp;</span>Evaluation Models with Time-series train test split</a></span></li><li><span><a href="#GridSearch-for-the-hyperparameter-for-KNN-and-Logistic-Regression" data-toc-modified-id="GridSearch-for-the-hyperparameter-for-KNN-and-Logistic-Regression-1.4.2"><span class="toc-item-num">1.4.2&nbsp;&nbsp;</span>GridSearch for the hyperparameter for KNN and Logistic Regression</a></span></li></ul></li><li><span><a href="#Gridsearch-to-find-the-best-parameters" data-toc-modified-id="Gridsearch-to-find-the-best-parameters-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Gridsearch to find the best parameters</a></span></li></ul></li></ul></div>

In [122]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
from sklearn.model_selection import train_test_split
import gensim
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import TaggedDocument
import seaborn as sns
import matplotlib.pyplot as plt
import regex as re

from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk

from datetime import date
import time
now = time.time()

In [233]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import MultinomialNB

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

from sklearn import svm, datasets
from sklearn.metrics import accuracy_score
from pactools.grid_search import GridSearchCVProgressBar

from sklearn.pipeline import Pipeline 

import warnings
warnings.filterwarnings('ignore')

## 1. Clean Up Datafame 

In [451]:
#read in dataframe 
df = pd.read_csv('../data/df_balanced_consolidated.csv')

In [469]:
def clean_df(df): 
    index_1 = df[df['publishedAt'].isnull()].index
    
    df = df.drop(index_1)
    df.reset_index(inplace = True, drop= True)
    
    boolian = []
    for i in range(len(df['publishedAt'])):
        boolian.append(df['publishedAt'][i][0] != '2')
    
    df = df.drop(df[boolian].index)
    df.reset_index(inplace = True, drop= True)
    
    df['date'] = df['publishedAt'].apply(lambda x: str(x).split('T')[0])
    df['date'] = pd.to_datetime(df['date'])
    df = df.set_index(df['date'])
    
    return df

In [470]:
df = clean_df(df)

In [472]:
df.head()

,author,content,description,publishedAt,source,source_id,source_name,title,types,url,urlToImage,yes_disaster,date
date,,,,,,,,,,,,,
2019-01-04,https://www.facebook.com/bbcnews,image copyrightepaimage caption tens of thous...,Storm Pabuk has made landfall in southern Thai...,2019-01-04T07:28:56Z,"{'id': 'bbc-news', 'name': 'BBC News'}",bbc-news,BBC News,Thailand Pabuk: Tourist spots hit by worst sto...,landfall,https://www.bbc.co.uk/news/world-asia-46756239,https://ichef.bbci.co.uk/news/1024/branded_new...,1,2019-01-04
2018-10-11,NEETI UPADHYE,1 u s paul manaforts trail of scandals,Hurricane Michael made landfall in Florida at ...,2018-10-11T16:18:46Z,"{'id': 'the-new-york-times', 'name': 'The New ...",the-new-york-times,The New York Times,‘Trees Flying By’: The Aftermath of Hurricane ...,landfall,https://www.nytimes.com/video/us/politics/1000...,https://static01.nyt.com/images/2018/10/12/us/...,1,2018-10-11
2018-10-12,Dell Cameron,the federal communications commission on frida...,The Federal Communications Commission on Frida...,2018-10-12T22:25:00Z,"{'id': None, 'name': 'Gizmodo.com'}",NaN,Gizmodo.com,"Widespread Cable, Internet, and Radio Outages ...",landfall,https://gizmodo.com/widespread-cable-internet-...,https://i.kinja-img.com/gawker-media/image/upl...,1,2018-10-12
2018-09-05,Reuters Editorial,reuters - tropical storm gordon was making l...,Tropical storm Gordon was making landfall just...,2018-09-05T03:12:45Z,"{'id': 'reuters', 'name': 'Reuters'}",reuters,Reuters,Storm Gordon making landfall west of Alabama-M...,landfall,https://www.reuters.com/article/us-storm-gordo...,https://s4.reutersmedia.net/resources_v2/image...,1,2018-09-05
2018-05-04,Ed Mazza,stephen colbert said former new york city mayo...,"Today's ""Stormy Watch"" finds Hurricane Rudy ""m...",2018-05-04T02:14:53Z,"{'id': 'the-huffington-post', 'name': 'The Huf...",the-huffington-post,The Huffington Post,Colbert Mocks Giuliani’s Bonkers Fox News Inte...,landfall,https://www.huffingtonpost.com/entry/stephen-c...,https://img.huffingtonpost.com/asset/5aebbeac1...,1,2018-05-04


In [475]:
df_train = df[['content', 'yes_disaster']]

In [476]:
df_train.head()

,content,yes_disaster
date,,
2019-01-04,image copyrightepaimage caption tens of thous...,1
2018-10-11,1 u s paul manaforts trail of scandals,1
2018-10-12,the federal communications commission on frida...,1
2018-09-05,reuters - tropical storm gordon was making l...,1
2018-05-04,stephen colbert said former new york city mayo...,1


In [474]:
df_train.isnull().sum()

content         0
yes_disaster    0
date            0
dtype: int64

###  Examing Content 

In [541]:
df_train['content'][10]

' reuters  - hurricane michael is strengthening and is expected to be a major hurricane when it makes landfall in northwestern florida on wednesday, the u s  national hurricane center  nhc  said in its latest advisory on the storm on tuesday  major hurricanes are those of category 3 and above on the on the five-step saffir-simpson scale  at 2 p m  et  1800 gmt , the center of michael was located about 310 miles  500 km  south-southwest of apalachicola, florida, the nhc said  it was carrying maximum sustained winds of 110 miles per hour  175 km per hour , the center said  the storm was moving north over the eastern gulf of mexico   life-threatening storm surge, hurricane force winds, and heavy rainfall expected along the northeastern gulf coast,  the miami-based forecaster said  reporting by karen rodrigues and k  sathya narayanan in bengaluru; editing by frances kerry'

## Text Processing

In [542]:
def clean_text(text): 
    '''
    This clean_text function will focus on: 
    1. cleaning the content including removing prentecisis, \r\n, : and so on. 
    2. lower case all words.
    3. remove stop words from articles. 
    '''
    text = re.sub(r'\r\n', r' ', text)
    text = re.sub(r'[\\\.\:\*/]', r' ', text)
    text = re.sub(r'[\(\)]', r' ', text)
    text = re.sub(r'[\"\“\”\—\[\]]', r' ', text)
    text = re.sub(r"'s", r' ', text)
    text = re.sub(r'[\\"]', r' ', text)
    text = text.lower()
    
    return text

In [545]:
clean_text(df_train['content'][10])

' reuters  - hurricane michael is strengthening and is expected to be a major hurricane when it makes landfall in northwestern florida on wednesday, the u s  national hurricane center  nhc  said in its latest advisory on the storm on tuesday  major hurricanes are those of category 3 and above on the on the five-step saffir-simpson scale  at 2 p m  et  1800 gmt , the center of michael was located about 310 miles  500 km  south-southwest of apalachicola, florida, the nhc said  it was carrying maximum sustained winds of 110 miles per hour  175 km per hour , the center said  the storm was moving north over the eastern gulf of mexico   life-threatening storm surge, hurricane force winds, and heavy rainfall expected along the northeastern gulf coast,  the miami-based forecaster said  reporting by karen rodrigues and k  sathya narayanan in bengaluru; editing by frances kerry'

In [546]:
df_train['content'] = df_train['content'].apply(clean_text)

In [547]:
df_train['content'][3]

' reuters  - tropical storm gordon was making landfall just west of the alabama-mississippi border, the u s  national hurricane center  nhc  said in its latest advisory on tuesday  gordon was located about 35 miles  55 kilometers  south-southwest of mobile, alabama and was packing maximum sustained winds of 70 miles per hour  110 km h , the miami-based weather forecaster said   rapid weakening is forecast after gordon moves inland, and is forecast to become a tropical depression on wednesday,  the nhc added reporting by nallur sethuraman and vijaykumar vedala in bengaluru; editing by joseph radford'

In [654]:
import pickle
custom_stop_words_final = pickle.load( open( "save.p", "rb" ) )

In [655]:
def lowercase(lst): 
    for i in range(len(lst)): 
        lst[i] = lst[i].lower()
        
    return lst

In [656]:
custom_stop_words_final = lowercase(custom_stop_words_final)

In [657]:
len(custom_stop_words_final)

1677

In [684]:
def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if word in custom_stop_words_final: 
                continue 
            if len(word) == 1: 
                continue 
            tokens.append(word.lower())
    return tokens

## Regular Train Test Split 

In [685]:
#train test split the data and tag Documents 
train, test = train_test_split(df_train, test_size=0.3, random_state=42)

In [686]:
train['yes_disaster'].value_counts()

0    3722
1    3721
Name: yes_disaster, dtype: int64

In [687]:
test['yes_disaster'].value_counts()

1    1616
0    1575
Name: yes_disaster, dtype: int64

In [688]:
%%time
train_tagged = train.apply(
    lambda row: TaggedDocument(words=tokenize_text(row['content']), tags=[row.yes_disaster]), axis=1)

test_tagged = test.apply(
    lambda row: TaggedDocument(words=tokenize_text(row['content']), tags=[row.yes_disaster]), axis=1)

CPU times: user 1min 50s, sys: 590 ms, total: 1min 50s
Wall time: 1min 51s


In [689]:
train_tagged.values[1]

TaggedDocument(words=['genoa', 'italy', 'rescuers', 'searched', 'night', 'survivors', 'shattered', 'remains', 'motorway', 'bridge', 'genoa', 'investigators', 'determine', 'caused', 'catastrophic', 'collapse', '30', 'people', 'killed', 'tuesday', 'vast', 'span', 'morandi', 'bridge', 'collapsed', 'heavy', 'rainstorm', 'sending', 'vehicles', 'drivers', 'plunging', '100-meters', 'railway', 'tracks', 'rescuers', 'spent', 'wednesday', 'dawn', 'searching', 'tangled', 'remains', 'bridge', 'floodlights', 'fears', 'toll', 'rise', 'government', 'called', 'immense', 'tragedy', 'collapse', 'came', 'bridge', 'undergoing', 'maintenance', 'work', 'liguria', 'region', 'genoa', 'situated', 'experienced', 'torrential', 'rainfall', 'unfortunately', '30', 'dead', 'injured', 'condition', 'interior', 'minister', 'matteo', 'salvini', 'told', 'reporters', 'president', 'sergio', 'mattarella', 'said', 'catastrophe', 'hit', 'genoa', 'italy', 'attention', 'turned', 'caused', 'collapse', 'ultimately', 'responsible'

In [690]:
test_tagged.values[10]

TaggedDocument(words=['delays', 'caused', 'drawn-out', 'negotiations', 'nonprofit', 'providers', 'city', 'tries', 'reel', 'rates', 'long', 'inconsistent', 'mr', 'banks', 'said', 'city', 'contended', 'nuts', 'bolts', 'construction', 'pointing', 'unopened', 'landing', 'road', 'shelter', 'alarm', 'control', 'panel', 'wrong', 'place', 'inherent', 'challenges', 'developing', 'opening', 'buildings', 'new', 'city', 'said', 'obstacle', 'speaks', 'heart', 'plan', 'plea', 'empathy', 'steady', 'community', 'resistance', 'shelters', 'men', 'legal', 'challenge', 'mayor', 'plan', 'came', 'opposition', 'shelter', '104', 'elderly', 'men', 'crown', 'heights', 'city', 'prevailed', 'shelter', 'opening', 'delayed', 'months', 'shack', 'chief', 'executive', 'urban', 'pathways', 'shelter', 'provider', 'said', 'city', 'benefit', 'public', 'education', 'campaign', 'dispel', 'stereotypes', 'homeless', 'men', 'data', 'shows', 'homeless', 'people', 'street', 'likely', 'victims', 'crime', 'perpetrators', 'crime', 

### Set-up Doc2Vec Training with Stopwords 

In [691]:
import multiprocessing
cores = multiprocessing.cpu_count()

In [692]:
cores

4

In [693]:
#building vocabulary model
model_dbow = Doc2Vec(dm=0, 
                     vector_size= 100, 
                     window = 15, 
                     negative=10, 
                     hs=0, 
                     min_count=5, 
                     sample = 0, 
                     workers=cores)
        
model_dbow.build_vocab([x for x in tqdm(train_tagged.values)])

100%|██████████| 7443/7443 [00:00<00:00, 1157558.85it/s]


In [695]:
%%time
for epoch in range(7):
    model_dbow.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), 
                     total_examples=len(train_tagged.values), epochs=1)
    model_dbow.alpha -= 0.001
    model_dbow.min_alpha = model_dbow.alpha

100%|██████████| 7443/7443 [00:00<00:00, 2750744.97it/s]


CPU times: user 51 s, sys: 488 ms, total: 51.5 s
Wall time: 16.4 s


In [696]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, regressors

In [697]:
%%time
y_train, X_train = vec_for_learning(model_dbow, train_tagged)
y_test, X_test = vec_for_learning(model_dbow, test_tagged)

CPU times: user 2min 21s, sys: 569 ms, total: 2min 22s
Wall time: 2min 23s


###  Evaluation Models

In [706]:
# Instantiate all of the models, and create lists of models and modelnames (strings)

lr  = LogisticRegression(solver='liblinear', multi_class='auto')

dt  = DecisionTreeClassifier(max_depth    = None,
                             min_samples_split = 2,
                             max_features = None,
                             random_state = 42)

bag = BaggingClassifier(n_estimators= 100)

rf  = RandomForestClassifier(n_estimators= 100, max_features=100, random_state = 42)

et  = ExtraTreesClassifier(n_estimators= 100, random_state = 42)

ab  = AdaBoostClassifier(n_estimators= 1000)

gb  = GradientBoostingClassifier(n_estimators=1000)

svc = svm.SVC()

knn = KNeighborsClassifier(n_neighbors= 100)


models_set1     = [ lr,   dt,   bag,   rf,   et]
modelnames_set1 = ['lr', 'dt', 'bag', 'rf', 'et']

models_set2     = [ ab,   gb,   svc ,  knn ]
modelnames_set2 = ['ab', 'gb', 'svc', 'knn']

In [707]:
#  Define a function to fit and score any instantiated model
#  Parameters: instantiated model, modelname(string), X_train, X_test, y_train, y_test
#  Returns:    train score, test score, overfit amt, fit time

def fit_score(model, modelname, X_train, X_test, y_train, y_test):
    
    time_start = time.time()
    model.fit(X_train, y_train)
    time_end   = time.time()
    fit_time   = time_end - time_start
    
    model_train_score = model.score(X_train, y_train)
    model_test_score  = model.score(X_test, y_test)
    overfit_amt       = model_train_score - model_test_score
    
    print(f'\n{modelname.upper()} fit time: {fit_time:.2f} sec')
    print(f'Train score: {model_train_score:.6f}  \
    Test score: {model_test_score:.6f}  \
    Overfit: {overfit_amt:.4f}')
    
    return model_train_score, model_test_score, overfit_amt, fit_time

In [708]:
# Round 1
# code from Manu
# use a for loop to call fit_score function on all nine models
# dictionary technique adapted from StackOverflow

results_dict = {}
for model, modelname in zip(models_set1, modelnames_set1):
    temp1, temp2, temp3, temp4 = fit_score(
                                            model, modelname,
                                            X_train, X_test,
                                            y_train, y_test)
    results_dict[modelname] = {}
    results_dict[modelname]['train_score'] = temp1
    results_dict[modelname]['test_score']  = temp2
    results_dict[modelname]['overfit_amt'] = temp3
    results_dict[modelname]['fit_time']    = temp4


LR fit time: 0.23 sec
Train score: 0.839446      Test score: 0.788154      Overfit: 0.0513

DT fit time: 0.88 sec
Train score: 1.000000      Test score: 0.667502      Overfit: 0.3325

BAG fit time: 54.05 sec
Train score: 1.000000      Test score: 0.781573      Overfit: 0.2184

RF fit time: 56.01 sec
Train score: 1.000000      Test score: 0.776872      Overfit: 0.2231

ET fit time: 1.23 sec
Train score: 1.000000      Test score: 0.783453      Overfit: 0.2165


In [709]:
results_df1 = pd.DataFrame(results_dict)
results_table1 = results_df1.T
results_table1

,fit_time,overfit_amt,test_score,train_score
lr,0.226733,0.051292,0.788154,0.839446
dt,0.884789,0.332498,0.667502,1.000000
bag,54.047184,0.218427,0.781573,1.000000
rf,56.007343,0.223128,0.776872,1.000000
et,1.234127,0.216547,0.783453,1.000000


In [710]:
# Round 2
# code from Manu
# use a for loop to call fit_score function on all nine models
# dictionary technique adapted from StackOverflow

results_dict = {}
for model, modelname in zip(models_set2, modelnames_set2):
    temp1, temp2, temp3, temp4 = fit_score(
                                            model, modelname,
                                            X_train, X_test,
                                            y_train, y_test)
    results_dict[modelname] = {}
    results_dict[modelname]['train_score'] = temp1
    results_dict[modelname]['test_score']  = temp2
    results_dict[modelname]['overfit_amt'] = temp3
    results_dict[modelname]['fit_time']    = temp4


AB fit time: 82.29 sec
Train score: 0.921000      Test score: 0.756816      Overfit: 0.1642

GB fit time: 61.36 sec
Train score: 0.993685      Test score: 0.782200      Overfit: 0.2115

SVC fit time: 6.81 sec
Train score: 0.865243      Test score: 0.795362      Overfit: 0.0699

KNN fit time: 0.03 sec
Train score: 0.790676      Test score: 0.754936      Overfit: 0.0357


In [711]:
results_df2 = pd.DataFrame(results_dict)
results_table2 = results_df2.T
results_table2

,fit_time,overfit_amt,test_score,train_score
ab,82.288448,0.164184,0.756816,0.921000
gb,61.360885,0.211485,0.782200,0.993685
svc,6.808115,0.069881,0.795362,0.865243
knn,0.028084,0.035740,0.754936,0.790676


In [712]:
lr.coef_

array([[-1.05398109e+00, -4.19707072e-01, -1.55357708e-01,
         1.86733531e-01, -3.78406686e-01, -2.12960114e-03,
        -3.70778395e-01,  1.78435492e-01,  3.85730247e-01,
         2.69375798e-01,  3.56583024e-01,  3.62967668e-01,
        -2.82888650e-01, -2.47805072e-02, -1.59810551e-01,
         4.09249568e-01,  2.51200497e-01, -2.81464073e-01,
        -6.90969018e-02,  1.75199775e-01, -3.44157278e-03,
        -1.01338714e-01, -2.45023596e-01, -3.98546962e-01,
         6.33128904e-01, -3.82032631e-01, -4.16819017e-01,
        -1.22501754e-01, -7.03152288e-02, -6.91133873e-02,
         1.87552217e-01,  9.63241756e-03,  1.26774454e-01,
         2.54359997e-01,  8.80700946e-02,  6.01290879e-01,
        -4.41115998e-01, -4.68605717e-01,  5.40738027e-01,
        -4.64032028e-02, -1.48123523e-01, -2.95351227e-01,
         2.92114043e-01, -3.05020018e-01, -3.56972348e-01,
         2.93084029e-01,  1.04505475e-01, -2.64743274e-01,
        -3.83546849e-01, -3.01806359e-02, -4.02014680e-0

## Set-up Doc2Vec Training with Stopwords and Time-series Train Test Split

In [713]:
print('2017 data shape', df_train['2017'].shape)

df_train['2017']['yes_disaster'].value_counts()

2017 data shape (1013, 2)


1    512
0    501
Name: yes_disaster, dtype: int64

In [714]:
from sklearn.utils import shuffle

In [715]:
print('2018 data shape', df_train['2018'].shape)

df_train['2018']['yes_disaster'].value_counts()

2018 data shape (7758, 2)


1    4006
0    3752
Name: yes_disaster, dtype: int64

In [716]:
df_train_2018 = shuffle(df_train['2018'])

In [717]:
print('2019 data shape', df_train['2019'].shape)

df_train['2019']['yes_disaster'].value_counts()

2019 data shape (1863, 2)


0    1044
1     819
Name: yes_disaster, dtype: int64

In [718]:
#train data includes: 2017, and 6000 rows from 2018 
train_df = pd.concat([df_train['2017'], df_train_2018[:5000]])
train_df['yes_disaster'].value_counts()

1    3118
0    2895
Name: yes_disaster, dtype: int64

In [719]:
train_df.head()

,content,yes_disaster
date,,
2017-12-21,five days after hurricane harvey slammed into ...,1
2017-12-13,federal reserve officials followed through on ...,1
2017-11-13,for developing communities in many parts of th...,1
2017-11-27,drivers are poised to pay the highest year-end...,1
2017-12-04,bob bentz had just installed underground utili...,1


In [720]:
train_df.to_csv('../data/train_csv')

In [721]:
#test data includes: 2018, and 2019
test_df = pd.concat([df_train_2018[5000:], df_train['2019']])
test_df['yes_disaster'].value_counts()

0    2402
1    2219
Name: yes_disaster, dtype: int64

In [722]:
test_df.head()

,content,yes_disaster
date,,
2018-04-23,now playing unruly passenger removed from ame...,1
2018-03-16,the dinner bell is ringing for the trade bar ...,0
2018-02-05,day-and-date announce and release is a new wri...,0
2018-03-14,"cambridge, vt an avalanche has hit vermon...",1
2018-05-17,the scientists examined precipitation trends a...,1


In [723]:
test_df.to_csv('../data/test_csv')

In [724]:
%%time
train_tagged2 = train_df.apply(
    lambda row: TaggedDocument(words=tokenize_text(row['content']), tags=[row.yes_disaster]), axis=1)

test_tagged2 = test_df.apply(
    lambda row: TaggedDocument(words=tokenize_text(row['content']), tags=[row.yes_disaster]), axis=1)

CPU times: user 1min 49s, sys: 565 ms, total: 1min 50s
Wall time: 1min 51s


In [652]:
train_tagged2[20]

TaggedDocument(words=['students', 'participate', 'training', 'program', 'shelter', 'earthquake', 'school', 'seoul', 'south', 'korea', 'nov', '16', '2017', 'photo', 'yonhap', 'epa-efe', 'seoul', 'south', 'korea', 'magnitude', 'earthquake', 'south', 'korea', 'second-strongest', 'decades', 'damaged', 'infrastructure', 'injured', 'dozens', 'people', 'left', '1,500', 'homeless', 'officials', 'said', 'thursday', 'deaths', 'reported', 'quake', 'rattled', 'southeastern', 'coastal', 'region', 'port', 'city', 'pohang', 'wednesday', 'afternoon', 'thursday', 'morning', '1,536', 'people', 'forced', 'evacuate', 'homes', '57', 'people', 'injured', 'ministry', 'interior', 'safety', 'said', 'statement', '1,000', 'houses', 'dozens', 'buildings', 'cars', 'damaged', 'destroyed', 'cracks', 'damage', 'military', 'facilities', 'bridges', 'port', 'facilities', 'water', 'supply', 'facilities', 'media', 'images', 'showed', 'crumbled', 'walls', 'piled', 'damaged', 'cars', 'broken', 'windows', 'cracks', 'building

In [631]:
#building vocabulary model
model_dbow2 = Doc2Vec(dm=0, 
                     vector_size= 100, 
                     window = 15, 
                     negative=10, 
                     hs=0, 
                     min_count=5, 
                     sample = 0, 
                     workers=cores)
        
model_dbow2.build_vocab([x for x in tqdm(train_tagged.values)])

100%|██████████| 7013/7013 [00:00<00:00, 1425681.17it/s]


In [632]:
%%time
for epoch in range(7):
    model_dbow2.train(utils.shuffle([x for x in tqdm(train_tagged2.values)]), 
                     total_examples=len(train_tagged2.values), epochs=1)
    model_dbow2.alpha -= 0.002
    model_dbow2.min_alpha = model_dbow2.alpha

100%|██████████| 6013/6013 [00:00<00:00, 2722110.09it/s]


CPU times: user 41.2 s, sys: 368 ms, total: 41.6 s
Wall time: 12.9 s


In [633]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, regressors

In [634]:
%%time
y_train, X_train = vec_for_learning(model_dbow2, train_tagged2)
y_test, X_test = vec_for_learning(model_dbow2, test_tagged2)

CPU times: user 2min 17s, sys: 543 ms, total: 2min 18s
Wall time: 2min 19s


###  Evaluation Models with Time-series train test split

In [725]:
# Instantiate all of the models, and create lists of models and modelnames (strings)

lr  = LogisticRegression(solver='liblinear', multi_class='auto')

dt  = DecisionTreeClassifier(max_depth    = None,
                             min_samples_split = 2,
                             max_features = None,
                             random_state = 42)

bag = BaggingClassifier(n_estimators= 100)

rf  = RandomForestClassifier(n_estimators= 100, max_features=100, random_state = 42)

et  = ExtraTreesClassifier(n_estimators= 100, random_state = 42)

ab  = AdaBoostClassifier(n_estimators= 1000)

gb  = GradientBoostingClassifier(n_estimators=1000)

svc = svm.SVC()

knn = KNeighborsClassifier(n_neighbors= 100)


models_set1     = [ lr,   dt,   bag,   rf,   et]
modelnames_set1 = ['lr', 'dt', 'bag', 'rf', 'et']

models_set2     = [ ab,   gb,   svc ,  knn ]
modelnames_set2 = ['ab', 'gb', 'svc', 'knn']

In [726]:
#  Define a function to fit and score any instantiated model
#  Parameters: instantiated model, modelname(string), X_train, X_test, y_train, y_test
#  Returns:    train score, test score, overfit amt, fit time

def fit_score(model, modelname, X_train, X_test, y_train, y_test):
    
    time_start = time.time()
    model.fit(X_train, y_train)
    time_end   = time.time()
    fit_time   = time_end - time_start
    
    model_train_score = model.score(X_train, y_train)
    model_test_score  = model.score(X_test, y_test)
    overfit_amt       = model_train_score - model_test_score
    
    print(f'\n{modelname.upper()} fit time: {fit_time:.2f} sec')
    print(f'Train score: {model_train_score:.6f}  \
    Test score: {model_test_score:.6f}  \
    Overfit: {overfit_amt:.4f}')
    
    return model_train_score, model_test_score, overfit_amt, fit_time

In [727]:
# Round 1
# code from Manu
# use a for loop to call fit_score function on all nine models
# dictionary technique adapted from StackOverflow

results_dict = {}
for model, modelname in zip(models_set1, modelnames_set1):
    temp1, temp2, temp3, temp4 = fit_score(
                                            model, modelname,
                                            X_train, X_test,
                                            y_train, y_test)
    results_dict[modelname] = {}
    results_dict[modelname]['train_score'] = temp1
    results_dict[modelname]['test_score']  = temp2
    results_dict[modelname]['overfit_amt'] = temp3
    results_dict[modelname]['fit_time']    = temp4


LR fit time: 0.22 sec
Train score: 0.839446      Test score: 0.788154      Overfit: 0.0513

DT fit time: 0.89 sec
Train score: 1.000000      Test score: 0.667502      Overfit: 0.3325

BAG fit time: 58.04 sec
Train score: 1.000000      Test score: 0.780946      Overfit: 0.2191

RF fit time: 57.34 sec
Train score: 1.000000      Test score: 0.776872      Overfit: 0.2231

ET fit time: 1.20 sec
Train score: 1.000000      Test score: 0.783453      Overfit: 0.2165


In [728]:
results_df1 = pd.DataFrame(results_dict)
results_table1 = results_df1.T
results_table1

,fit_time,overfit_amt,test_score,train_score
lr,0.218981,0.051292,0.788154,0.839446
dt,0.892328,0.332498,0.667502,1.000000
bag,58.040239,0.219054,0.780946,1.000000
rf,57.343921,0.223128,0.776872,1.000000
et,1.200405,0.216547,0.783453,1.000000


In [729]:
# Round 2
# code from Manu
# use a for loop to call fit_score function on all nine models
# dictionary technique adapted from StackOverflow

results_dict = {}
for model, modelname in zip(models_set2, modelnames_set2):
    temp1, temp2, temp3, temp4 = fit_score(
                                            model, modelname,
                                            X_train, X_test,
                                            y_train, y_test)
    results_dict[modelname] = {}
    results_dict[modelname]['train_score'] = temp1
    results_dict[modelname]['test_score']  = temp2
    results_dict[modelname]['overfit_amt'] = temp3
    results_dict[modelname]['fit_time']    = temp4


AB fit time: 79.99 sec
Train score: 0.921000      Test score: 0.756816      Overfit: 0.1642

GB fit time: 61.00 sec
Train score: 0.993685      Test score: 0.782200      Overfit: 0.2115

SVC fit time: 5.17 sec
Train score: 0.865243      Test score: 0.795362      Overfit: 0.0699

KNN fit time: 0.02 sec
Train score: 0.790676      Test score: 0.754936      Overfit: 0.0357


In [731]:
results_df2 = pd.DataFrame(results_dict)
results_table2 = results_df2.T
results_table2

,fit_time,overfit_amt,test_score,train_score
ab,79.992350,0.164184,0.756816,0.921000
gb,61.001763,0.211485,0.782200,0.993685
svc,5.169062,0.069881,0.795362,0.865243
knn,0.021383,0.035740,0.754936,0.790676


### GridSearch for the hyperparameter for KNN and Logistic Regression 

In [732]:
loggit_pipe = Pipeline([
    ('logreg', LogisticRegression())
])

#parameters for grid search 
params = {
    'logreg__C': [0.001,0.01,0.1,1,10,100,1000],
    'logreg__solver' : ['liblinear', 'sag']    
}

#gridsearch 
gs = GridSearchCV(loggit_pipe, param_grid = params , cv = 5)
gs.fit(X_train, y_train)
gs.best_params_

{'logreg__C': 1, 'logreg__solver': 'liblinear'}

In [733]:
print('train score', gs.score(X_train, y_train))
print('test score', gs.score(X_test, y_test))

train score 0.8394464597608491
test score 0.7881541836414917


In [734]:
%%time 
#KNN 
knn_pipe = Pipeline([
    ('knn', KNeighborsClassifier())
])

#parameters for grid search 
params = {
    'knn__n_neighbors': [50,100,150, 200, 250],
    'knn__weights' : ['uniform', 'distance']    
}

#gridsearch 
gs_knn = GridSearchCV(knn_pipe, param_grid = params , cv = 5)
gs_knn.fit(X_train, y_train)
gs_knn.best_params_

CPU times: user 7min 1s, sys: 2.9 s, total: 7min 4s
Wall time: 7min 6s


In [735]:
gs_knn.best_params_

{'knn__n_neighbors': 150, 'knn__weights': 'uniform'}

In [736]:
print('train score', gs_knn.score(X_train, y_train))
print('test score', gs_knn.score(X_test, y_test))

train score 0.791347574902593
test score 0.7649639611407082


## Gridsearch to find the best parameters

In [215]:
%%time

# #gridsearch doc2vec 
# sizes = [100,200,300]
# windows = [5,10,15,20]
# epochs = [5,8,10,15]
# summary_list = []

# for size in sizes: 
#     for window in windows: 
#         for epoch in epochs: 
#             model_dbow = Doc2Vec(dm=0, 
#                      vector_size=size, 
#                      window = window, 
#                      negative=10, 
#                      hs=0, 
#                      min_count=5, 
#                      sample = 0, 
#                      workers=cores)
        
#             model_dbow.build_vocab([x for x in tqdm(train_tagged.values)])

#             for epoch in range(epoch):
#                 model_dbow.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), 
#                 total_examples=len(train_tagged.values), epochs=1)
#                 model_dbow.alpha -= 0.001
#                 model_dbow.min_alpha = model_dbow.alpha
    
#             y_train, X_train = vec_for_learning(model_dbow, train_tagged)
#             y_test, X_test = vec_for_learning(model_dbow, test_tagged)
            
#             knn = KNeighborsClassifier(n_neighbors= 100)
#             knn.fit(X_train, y_train)
            
#             train_score = knn.score(X_train, y_train)
#             test_score = knn.score(X_test, y_test)
            
#             summary = {}
#             summary['size'] = size 
#             summary['window'] = window 
#             summary['epoch'] = epoch
#             summary['train_score']  = train_score
#             summary['test_score'] = test_score
            
#             summary_list.append(summary)
            
# summaries_dbow_df = pd.DataFrame(summary_list)
# summaries_dbow_df

100%|██████████| 7724/7724 [00:00<00:00, 1766263.44it/s]


100%|██████████| 7724/7724 [00:00<00:00, 1868004.62it/s]


CPU times: user 8h 23min 39s, sys: 1min 37s, total: 8h 25min 17s
Wall time: 7h 33min 31s


In [220]:
summaries_dbow_df.sort_values(by = ['test_score'], ascending= False)

,epoch,size,test_score,train_score,window
9,7,100,0.785563,0.807613,15
26,9,200,0.785563,0.814345,15
13,7,100,0.784657,0.808778,20
0,4,100,0.784053,0.803470,5
36,4,300,0.784053,0.807354,10
47,14,300,0.784053,0.816805,20
22,9,200,0.783751,0.811885,10
7,14,100,0.783147,0.811497,10
10,9,100,0.783147,0.813956,15
16,4,200,0.783147,0.803211,5
